In [16]:
import pickle
import re
import pdb

import pandas as pd
import numpy as np

from IPython.display import display

### Converting Raw API data to DataFrame

In [166]:
data = pd.read_json('data/raw_api_data.txt')

cols_to_parse = ['Gender', 'SeverityType', 'Locale', 'ProductCategory']
cols_to_add = [['GenderDescription','GenderId','GenderPublicName'],
['IncidentDetails','SeverityTypeDescription','SeverityTypePublicName'],
['LocaleDescription','LocalePublicName'],
['ProductCategoryDescription','ProductCategoryPublicName']]
new_df = pd.DataFrame()
cols_dict = dict(zip(cols_to_parse, cols_to_add))

for key in cols_dict:
    placeholder = pd.DataFrame(columns = cols_dict[key])
    for (i, row) in data.iterrows():
        e = row[key]
        value_holder = []
        for item in cols_dict[key]:
            try:
                component = e[item]
            except:
                component = 'Missing'
            value_holder.append(component)
        placeholder.loc[i, :] = value_holder
    if new_df.shape[0] == 0:
        new_df = placeholder
    else:
        new_df = pd.concat([new_df, placeholder], axis=1)
        
new_df2 = pd.concat([data, new_df], axis=1)
new_df2 = new_df2.drop(['CompanyComments', 'Gender','IncidentDocuments','IncidentDetails', 'Locale', 'ProductCategory',
                      'RelationshipType','SeverityType', 'SourceType'], axis=1)
new_df2.to_pickle('data/cleaned_api_data')

### Analysis

In [17]:
data = pickle.load( open("data/cleaned_api_data", "rb" ) )
neiss = pd.read_csv('data/NEISS-data-2015-updated-APRIL2016.csv')

In [18]:
display(data.head())

,AnswerExplanation,CompanyCommentsExpanded,IncidentDate,IncidentDescription,IncidentProductDescription,IncidentReportDate,IncidentReportId,IncidentReportNumber,IncidentReportPublicationDate,LocaleId,...,__metadata,LocaleDescription,LocalePublicName,GenderDescription,GenderId,GenderPublicName,ProductCategoryDescription,ProductCategoryPublicName,SeverityTypeDescription,SeverityTypePublicName
0,None,Helen of Troy acknowledges receipt of the subm...,/Date(1297036800000)/,Using the Revlon rv050 curling iron and came h...,rv050 curling iron,/Date(1299801600000)/,1170172,20110311-B3E19-2147481666,/Date(1301709487243)/,-1,...,"{u'type': u'CPSRMS_PUBModel.IncidentDetail', u...",Unspecified,Unspecified,Missing,Missing,Missing,"Hair Curlers, Curling Irons, Clips & Hairpins","Hair Curlers, Curling Irons, Clips & Hairpins",Missing,Missing
1,None,None,/Date(881884800000)/,"On December 12th 1997, I found my son, Tyler J...","ChildCraft drop side crib, oak",/Date(1299801600000)/,1170340,20110311-CFE0F-2147481661,/Date(1301674156573)/,1,...,"{u'type': u'CPSRMS_PUBModel.IncidentDetail', u...",Home/Apartment/Condominium,Home/Apartment/Condominium,Male,2,Male,Cribs,Cribs,Death,Death
2,I have contacted the manufacturer several time...,None,/Date(1294876800000)/,I have a Frigidaire electric range that comes ...,Electric Smoothtop Range,/Date(1299801600000)/,1170342,20110311-CFAB7-2147481658,/Date(1302051235430)/,-1,...,"{u'type': u'CPSRMS_PUBModel.IncidentDetail', u...",Unspecified,Unspecified,Missing,Missing,Missing,Electric Ranges or Ovens (Excl Counter-top Ovens),Electric Ranges or Ovens (Excl Counter-top Ovens),Missing,Missing
3,Sears wants $500 to fix this product. It is a...,Sears Holdings takes product safety issues ver...,/Date(1299801600000)/,Kenmore Elite Model #795.77543600\r\n\r\nThe l...,Kenmore Elite Trio 25Cubic Feet Bottom Freezer...,/Date(1299801600000)/,1170344,20110311-EFD8B-2147481655,/Date(1301709795607)/,1,...,"{u'type': u'CPSRMS_PUBModel.IncidentDetail', u...",Home/Apartment/Condominium,Home/Apartment/Condominium,Male,2,Male,Refrigerators,Refrigerators,Received care that did not involve medical per...,First Aid Received by Non-Medical Professional
4,I will be writing a letter to the company foll...,Thank you for sharing your experience with us ...,/Date(1299456000000)/,"Since he was born two months ago, we have been...","Pampers Swaddlers New Baby with Dry Max, Size 1-2",/Date(1299801600000)/,1170347,20110311-DBB63-2147481650,/Date(1301661238360)/,1,...,"{u'type': u'CPSRMS_PUBModel.IncidentDetail', u...",Home/Apartment/Condominium,Home/Apartment/Condominium,Male,2,Male,Diapers,Diapers,Received care that did not involve medical per...,First Aid Received by Non-Medical Professional


In [19]:
display(neiss.head())

,CPSC Case #,trmt_date,psu,weight,stratum,age,sex,race,race_other,diag,diag_other,body_part,disposition,location,fmv,prod1,prod2,narr1,narr2
0,150733174,7/11/15,38,15.7762,V,5,1,0,NaN,57,NaN,33,1,9,0,1267,NaN,5 YR OLD MALE PLAYING SOCCER BLOCKING BALL AND...,X IT
1,150734723,7/6/15,43,83.2157,S,36,1,1,NaN,57,NaN,34,1,1,0,1439,4057,"L WRIST FX(?)/36YOWM 2 WKS AGO@G-FRIENDS HM,MO...",AN OBJECT&TWISTED WRIST TRYING TO CONTOL MOWER...
2,150817487,8/2/15,51,74.8813,L,20,2,0,NaN,71,OTITIS EXTERNA,94,1,0,0,3274,NaN,20 YO F C/O EAR PAIN 1 DAY SAS WAS SWIMMING YE...,T TRIED TO USE ***S TO REMOVE WAX MADE IT WORS...
3,150717776,6/26/15,41,15.7762,V,61,1,0,NaN,71,PAIN,35,1,0,0,611,NaN,61YOM W/KNEE PAIN S/P FALLING IN SHOWER 2 DAYS...,NaN
4,150721694,7/4/15,42,74.8813,L,88,2,3,HISPANIC,62,NaN,75,1,0,0,1893,1807,88YOF PAIN TO HEAD WHEN FALL TO FLOOR WHEN WAL...,DOOR AND FELL.DENIES LOC. DX HEAD INJURY


#### Next Steps

Ideally, what we would want to do is take the product names and then parse them out so we can have a list of all the items from the API data. From there, since the NEISS data is mostly in long form and they don't have a specific product, we'd have to go a regex match to see if the item is mentioned in the narrative. That's going to be the first way to match up the incident rates by product

So far, I've worked on a prototype for parsing out objects, but it's still incomplete so any help would be greatly appreciated.